In [1]:
import numpy as np
import pandas as pd
import os
from math import log
from pylab import random
import time
from collections import Counter
from tqdm import tqdm


In [23]:
DATA_PATH = 'ntust-ir-2020_hw4_v2'
TOPIC = 8
ALPHA = 0.6
BETA = 0.01
MAX_ITER = 30
THRESH = 0.5

In [3]:
# return word vector
def open_files(root_path = DATA_PATH, extension = ".txt"):
    
    path_query = DATA_PATH + "/queries"
    path_docs = DATA_PATH + "/docs"
    
    qlf = open(os.path.join(DATA_PATH, "query_list.txt"))
    dlf = open(os.path.join(DATA_PATH, "doc_list.txt"))
    
    querys = {}
    for fname in qlf:
        fname = fname.strip("\n")
        file = os.path.join(path_query, fname + extension)
        
        fq = open(file)
        query = [q.strip('\n').lower().split(' ') for q in fq][0]
        querys[fname] = query
        fq.close()

    docs = {}
    for fname in dlf:
        fname = fname.strip("\n")
        file = os.path.join(path_docs, fname + extension)
        
        fd = open(file)
        doc = [d.strip("\n").lower().split(" ") for d in fd][0]
        docs[fname] = doc
        fd.close()

    dlf.close()
    qlf.close()

    return querys, docs

In [4]:
def word_vector(query_dict):
    query_terms = set()
    for fname, terms in query_dict.items():
        for t in terms:
            query_terms.add(t)
    
    return list(query_terms)

In [5]:
def term_frequency(WV, docs_dict):
        
    tf = {}
    for fname in docs_dict.keys():
        tf[fname] = {}
        for word in WV:
            tf[fname][word] = 0
    
    for fname, terms in docs_dict.items():
        for term in terms:
            try:
                tf[fname][term] += 1
            except:
                continue
    
    return tf

In [6]:
def P_bg(tf, docs_dict):
    
    total_doc_len = 0
    for terms in docs_dict.values():
        total_doc_len += len(terms)
    
    print(f'total doc len: {total_doc_len}')
    
    p_bg = {}
    for fname, terms in tf.items():
        for term, tf_ in terms.items():
            if term in p_bg:
                p_bg[term] += tf_ / total_doc_len
            else:
                p_bg[term] = tf_ / total_doc_len
    
    return p_bg

In [7]:
"""
return: word vector, tf, P(wi|BG)
"""
def preprocess(querys_dict, docs_dict):
    t1 = time.time()
    WV = word_vector(querys_dict)
    print(f'cost time: {time.time() - t1}')
    
    t1 = time.time()
    tf = term_frequency(WV, docs_dict)
    print(f'cost time: {time.time() - t1}')
    
    p_bg = P_bg(tf, docs_dict)
    
    return WV, tf, p_bg

In [8]:
def init_param(WV, docs):
    
    WORD = len(WV)
    DOC = len(docs)
    
    pem = random([TOPIC, DOC])
    pm = random([WORD, TOPIC])
    
    # P(wi|TK)
    PM = {}
    for w, word in enumerate(WV):
        norm = sum(pm[w, :])
        PM[word] = {}
        for t in range(TOPIC):
            pm[w, t] /= norm
            PM[word][t] = pm[w, t]
    
    # P(Tk|dj)
    PEM = {}    
    for t in range(TOPIC):
        norm = sum(pem[t, :])
        PEM[t] = {}
        for d, fname in enumerate(docs.keys()):
            pem[t, d] /= norm
            PEM[t][fname] = pem[t, d]
    
    
    # P(Tk|wi, dj)
    PE = {}    
    for t in range(TOPIC):
        PE[t] = {}
        for idw, word in enumerate(WV):
            PE[t][word] = {}
            for idd, fname in enumerate(docs.keys()):
                PE[t][word][fname] = 0
        
    return PEM, PE, PM

In [9]:
def E_step(PEM, PE, PM):
    for word in WV:
        for fname in docs:
            denom = 0
            for t in range(TOPIC):
                PE[t][word][fname] = PM[word][t] * PEM[t][fname]
                denom += PE[t][word][fname]
            for t in range(TOPIC):
                if denom == 0:
                    PE[t][word][fname] = 0
                else:
                    PE[t][word][fname] /= denom
    return PE

In [10]:
def M_step(PM, PE):
    for t in range(TOPIC):
        denom = 0
        for word in WV:
            PM[word][t] = 0
            for fname in docs:
                PM[word][t] += tf[fname][word] * PE[t][word][fname]
            denom += PM[word][t]
        if denom == 0:
            for word in WV:
                PM[word][t] = 1.0 / len(WV)
        else:
            for word in WV:
                PM[word][t] /= denom
    return PM

In [11]:
def EM(PEM, PE, PM):
    PE = E_step(PEM, PE, PM)
    PM = M_step(PM, PE)
    
    for fname in docs:
        for t in range(TOPIC):
            PEM[t][fname] = 0
            denom = 0
            for word in WV:
                PEM[t][fname] += tf[fname][word] * PE[t][word][fname]
                denom += tf[fname][word]
            if denom == 0:
                PEM[t][fname] = 1.0 / TOPIC
            else:
                PEM[t][fname] /= denom
    
    return PEM, PE, PM

In [12]:
# log likelihood
def PLSA():
    
    t1 = time.time()
    # PEM = P(Tk|dj), PE = P(Tk|wi, dj), PM = P(wi|TK)
    PEM, PE, PM = init_param(WV, docs)
    
    print(f'cost time: {time.time() - t1}')
    
    new_log = old_log = 1
    
    for i in tqdm(range(0, MAX_ITER)):
        PEM, PE, PM = EM(PEM, PE, PM)
        new_log = 0
        for fname in tqdm(docs):
            for word in WV:
                tmp = 0
                for t in range(TOPIC):
                    tmp += PM[word][t] * PEM[t][fname]
                if tmp > 0:
                    new_log += tf[fname][word] * log(tmp)
        
        print(new_log, old_log)
        if old_log != 1 and new_log - old_log < THRESH:
            break
        old_log = new_log
    
    return PM, PEM

In [13]:
def output(filename = "result.txt"):
    # output file
    if os.path.exists(filename):
        os.remove(filename)

    with open(filename, "w") as ofile:
        ofile.write("Query,RetrievedDocuments\n")
        for query_name, score_list in sorted_sim_dict.items():
            ofile.write(query_name + ",")
            for score in score_list:
                ofile.write(score + " ")
            ofile.write("\n")

In [14]:
querys, docs = open_files()

In [15]:
WV, tf, p_bg = preprocess(querys, docs)

cost time: 5.364418029785156e-05
cost time: 1.7009921073913574
total doc len: 7059938


In [16]:
t1 = time.time()
PM, PEM = PLSA()
print(time.time() - t1)

:07<00:00, 1797.84it/s]
 87%|████████▋ | 26/30 [28:16<04:33, 68.45s/it]-2027717.2838216173 -2029228.3329315793

 90%|█████████ | 27/30 [29:26<03:26, 68.87s/it]-2026404.2765719723 -2027717.2838216173

 93%|█████████▎| 28/30 [30:37<02:19, 69.51s/it]-2025211.9308869666 -2026404.2765719723

 97%|█████████▋| 29/30 [31:48<01:09, 69.80s/it]-2024153.4171297878 -2025211.9308869666

100%|██████████| 30/30 [32:59<00:00, 65.97s/it]-2023218.8622896334 -2024153.4171297878
1984.4789025783539



In [24]:
# Answer without EM step
sim_dict = {}
sorted_sim_dict = {}
for fqname, terms in querys.items():
    sim_dict[fqname] = {}
    for fdname, doc_terms in docs.items():
        sim_score = 1
        for term in terms:
            plsa = 0
            for t in range(TOPIC):
                plsa += PM[term][t] * PEM[t][fdname]
            sim_score *= ALPHA * tf[fdname][term] / len(doc_terms) + BETA * plsa + (1 - ALPHA - BETA) * p_bg[term]
            
        sim_dict[fqname][fdname] = sim_score
    sorted_sim_dict[fqname] = sorted(sim_dict[fqname], key=sim_dict[fqname].get, reverse=True)

In [25]:
output('output.txt')